In [29]:
import pandas as pd
import os

os.chdir(r'C:\Users\jf79\Repos\REAP-Analytics')
import myfuncs.myfuncs as mf

import warnings
warnings.filterwarnings('ignore')

cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data'
os.chdir(cwd)
mf.read_directory()

master_data = pd.read_csv('Netcall Interactions Master Data.csv',low_memory=False)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data
Files in: ['desktop.ini', 'Interactions Feb 2023 - Apr 2024 (excl Cust Serv)', 'Interactions Feb 2025 (excl Cust Serv)', 'Interactions Jan 2022 - Dec 2022 (excl Cust Serv)', 'Interactions Jan 2024 - Jun 2024 (excl Cust Serv)', 'Interactions Jun 24 - Jan 25 (excl Cust Serv)', 'Interactions_Jan 2023 (LIMITED GROUPS)', 'Netcall Dashboard Data.csv', 'Netcall Interactions Master Data.csv']



In [30]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

master_df = master_data.copy()

columns_to_drop = [
    'customername','assistedby',
    'casereference','ipaddress'
]
master_df.drop(columns=columns_to_drop, inplace=True)

master_df['nullcount'] = master_df.isna().sum(axis=1)
master_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(drop.index, inplace=True)
master_df.drop(columns='nullcount', inplace=True)

columns_to_change = ['timeended', 'timequeued']
for column in columns_to_change:
    master_df[column] = pd.to_datetime(master_df[column])

master_df['queuetime'] = master_df['timeended'] - master_df['timequeued']

columns_to_change = [
    'duration','talktime','holdtime',
    'wrapupduration','agenthandlingtime'
]
master_df[columns_to_change] = master_df[columns_to_change].apply(lambda x: x.str.replace(r'(?<!^)-', '', regex=True))
for column in columns_to_change:
    master_df[column] = pd.to_timedelta(master_df[column])


master_df['dateofqueue'] = master_df['timequeued'].dt.date
# master_df.loc[(master_df['type'] == 'Call') & (master_df['queuetime'] < pd.Timedelta(seconds=40)),'isunder40s?'] = 1
# master_df['isunder40s?'].fillna(0, inplace=True)
master_df

,interactionid,groupname,type,direction,timequeued,timeended,queue,skill,agent,customercontact,queuetime,duration,talktime,holdtime,wrapupduration,agenthandlingtime,result,activitycode,transferredto,subject,conversationstarttime,conversationendtime,conversationduration,agentusername,agentid,forwarded,source,conversationid,Year,Month,dateofqueue
0,2640671,Repair CSC,Email,Inbound,2022-01-01 02:15:00,2022-01-04 16:01:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,Gbenga.Osunkoya@lbhf.gov.uk,3 days 13:46:00,0 days 00:04:05,0 days 00:04:05,0 days 00:00:00,0 days 00:00:04,0 days 00:04:09,Replied,Repair Status,NaN,RE: Inspection item Requires Attention - (BLK) 1-180 Poynter House,01/01/2022 02:15,04/01/2022 16:01,85:46:28,4861,4861.0,0.0,Email,2516337.0,2022,1,2022-01-01
1485943,2640673,Customer Services,Email,Inbound,2022-01-01 03:05:38,2022-01-04 09:09:58,Electoral Services,Electoral Services Email,Gillian Hoyle,mccatigan@gmail.com,3 days 06:04:20,0 days 00:00:23,0 days 00:00:22,0 days 00:00:01,0 days 00:00:00,0 days 00:00:22,Other,NaN,NaN,Register to Vote,01/01/2022 03:05:38,04/01/2022 09:09:58,78:04:20,NaN,NaN,1.0,NaN,2516339.0,2022,1,2022-01-01
1,2640679,Repair CSC,Email,Inbound,2022-01-01 06:14:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Georgia St Rose,Baboucarr.Gaye@lbhf.gov.uk,3 days 09:51:00,0 days 00:03:35,0 days 00:03:34,0 days 00:00:01,0 days 00:00:09,0 days 00:03:43,Replied,Repair Status,NaN,RE: Norland House Electrical Failure (Job Ref. 1126163/1 emergency within 4hrs),01/01/2022 06:14,04/01/2022 16:05,81:51:20,4861,4861.0,0.0,Email,2516345.0,2022,1,2022-01-01
2,2640681,Repair CSC,Email,Inbound,2022-01-01 09:08:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Emma Hicks,orbisreports@orbisprotect.com,3 days 06:57:00,0 days 00:03:06,0 days 00:03:06,0 days 00:00:00,0 days 00:00:16,0 days 00:03:22,Transferred,OOH - Related,Mehul Patel,[Orbis Protect] - Unallocated Cases,01/01/2022 09:08,04/01/2022 16:17,79:09:21,971,971.0,0.0,Email,2516347.0,2022,1,2022-01-01
3,2640683,Repair CSC,Email,Inbound,2022-01-01 09:38:00,2022-01-04 16:19:00,Repairs customer e-mail,E-Mail Responses - Customer,Emma Hicks,noreply@lbhf.gov.uk,3 days 06:41:00,0 days 00:14:11,0 days 00:14:10,0 days 00:00:01,0 days 00:01:00,0 days 00:15:10,Replied,Repair Status,NaN,Form submission from: Housing repair notification form,01/01/2022 09:38,04/01/2022 16:19,78:41:00,971,971.0,0.0,Email,2516349.0,2022,1,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462502,7847277,Customer Services,Call,Inbound,2025-03-21 17:00:27,2025-03-21 17:07:16,CTAX Recovery,CTAX Recovery,Anne Baxter,07737998852,0 days 00:06:49,0 days 00:06:40,0 days 00:06:40,0 days 00:00:00,0 days 00:02:26,0 days 00:09:06,Answered,Resolved at first point of contact,NaN,NaN,21/03/2025 17:00:27,21/03/2025 17:07:16,00:06:49,NaN,NaN,NaN,NaN,7475321.0,2025,3,2025-03-21
2462503,7847279,Customer Services,Call,Inbound,2025-03-21 17:00:50,2025-03-21 17:04:19,Planning,Planning,Jamie Barrett,07977730717,0 days 00:03:29,0 days 00:03:14,0 days 00:03:14,0 days 00:00:00,0 days 00:00:04,0 days 00:03:18,Answered,Passed to service team,NaN,NaN,21/03/2025 17:00:50,21/03/2025 17:04:19,00:03:29,NaN,NaN,NaN,NaN,7475323.0,2025,3,2025-03-21
2462504,7847289,Customer Services,Email,Inbound,2025-03-21 17:02:25,2025-03-21 17:06:43,Agent to Agent Transfer,NaN,Joseph Addison,chris.twiselton@hamunitedcharities.com,0 days 00:04:18,0 days 00:04:18,0 days 00:04:18,0 days 00:00:00,0 days 00:00:00,0 days 00:04:18,Replied,NaN,NaN,"Flat 50 Sycamore House, W6 0AS â€“ Ronald Gurd",21/03/2025 10:05:59,21/03/2025 17:06:43,07:00:44,NaN,NaN,0.0,NaN,7470933.0,2025,3,2025-03-21
2462505,7847293,Customer Services,Call,Outbound,2025-03-21 17:03:57,2025-03-21 17:04:55,Cost of Living Crisis,Cost of Living Crisis,Joseph Addison,07751730530,0 days 00:00:58,0 days 00:00:53,0 days 00:0

In [31]:
aggregated_interactions = master_df.groupby(['direction','type','dateofqueue','groupname','queue','result']).agg(
    number_of_interactions = ('interactionid', 'count'),
    wrap_upduration = ('wrapupduration','sum'),
    queue_time = ('queuetime','sum'),
    duration = ('queuetime','sum'),
    talk_time = ('talktime','sum'),
    hold_time = ('holdtime','sum'),
    agent_handling_time = ('agenthandlingtime','sum'),
    # isunder40s = ('isunder40s?','sum')
).reset_index()

columns_to_change = [
    'wrap_upduration','queue_time','duration',
    'talk_time','hold_time','agent_handling_time'
]
for column in columns_to_change:
    aggregated_interactions[column].fillna(pd.Timedelta(0), inplace=True)
    aggregated_interactions[column] = aggregated_interactions[column].apply(lambda x: x.seconds)

mf.validate_data(aggregated_interactions)

#########################################################################################################################################################################################
DataFrame: None


,direction,type,dateofqueue,groupname,queue,result,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time
0,Inbound,Call,2022-01-04,ACE and Admissions,ACE and Admissions,Answered,15,4241,5580,5580,5349,0,9590
1,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Abandoned,2,0,240,240,0,0,0
2,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Answered,66,1954,25260,25260,23670,0,25624
3,Inbound,Call,2022-01-04,CAS Team 1 6945,CAS Team 1 6945 Queue,Answered,3,180,300,300,210,0,390
4,Inbound,Call,2022-01-04,CAS Team 2 6946,CAS Team 2 6946 Queue,Answered,3,180,360,360,295,0,475
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196291,Outbound,Email,2025-03-19,Customer Services,Cost of Living Crisis Email,Sent,3,180,377,377,376,1,556
196292,Outbound,Email,2025-03-20,Customer Services,Cost of Living Crisis Email,Sent,2,120,265,265,263,1,383
196293,Outbound,Email,2025-03-21,Customer Services,Cost of Living Crisis Email,Deleted,1,130,382,382,381,1,511
196294,Outbound,Email,2025-03-21,Customer Services,Cost of Living Crisis Email,Sent,1,60,203,203,203,0,263


Unique values per field:


,Field Name,No. of Unique Values
0,direction,2
1,type,3
2,dateofqueue,1169
3,groupname,47
4,queue,186
5,result,11
6,number_of_interactions,425
7,wrap_upduration,9250
8,queue_time,34050
9,duration,34050



Number of duplicate rows:
0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196296 entries, 0 to 196295
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   direction               196296 non-null  object
 1   type                    196296 non-null  object
 2   dateofqueue             196296 non-null  object
 3   groupname               196296 non-null  object
 4   queue                   196296 non-null  object
 5   result                  196296 non-null  object
 6   number_of_interactions  196296 non-null  int64 
 7   wrap_upduration         196296 non-null  int64 
 8   queue_time              196296 non-null  int64 
 9   duration                196296 non-null  int64 
 10  talk_time               196296 non-null  int64 
 11  hold_time               196296 non-null  int64 
 12  agent_handling_time     196296 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 19.5+ MB
DataFrame: d

None


Summary statistics:


,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time
count,196296.000000,196296.000000,196296.000000,196296.000000,196296.000000,196296.000000,196296.000000
mean,12.290301,725.759664,8351.467126,8351.467126,2939.015349,1670.109340,3656.495145
std,33.527249,3380.230590,16828.546516,16828.546516,7379.929579,8352.916457,8995.803583
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,282.000000,282.000000,0.000000,0.000000,0.000000
50%,3.000000,30.000000,1300.000000,1300.000000,334.000000,0.000000,465.000000
75%,11.000000,272.000000,6552.250000,6552.250000,2047.000000,108.000000,2634.000000
max,4154.000000,85098.000000,86399.000000,86399.000000,86391.000000,86399.000000,86388.000000


End of data validation
#########################################################################################################################################################################################



In [32]:
mf.export_to_csv(
    aggregated_interactions,
    directory = r'C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data',
    df_name='Netcall Dashboard Data'
)

Exproting Netcall Dashboard Data to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Interactions Data\Netcall Dashboard Data.csv

Successfully exported Netcall Dashboard Data to CSV
